In [1]:
# ============================================================================
# 🏆 LLM Classification Finetuning - 선호도 예측
# ============================================================================
# **목표**: 두 개의 LLM 응답 중 사용자가 선호하는 것을 예측
# **평가 지표**: Log Loss
# **접근 방법**:
#   1. 어휘적 특징 (길이, 단어 수, 문장부호 등)
#   2. Sentence Transformers 임베딩
#   3. 5-Fold 교차 검증
#   4. Logistic Regression

# ============================================================================
# 📦 Cell 1: 라이브러리 및 설정
# ============================================================================
# ==== 환경 플래그 (가장 위!) ====
import os
os.environ["TRANSFORMERS_DISABLE_CHAT_TEMPLATE_AUTO_CONVERSION"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["TRANSFORMERS_OFFLINE"] = "1"       # ✅ 허브 조회 자체 차단
os.environ["HF_HUB_OFFLINE"] = "1"             # ✅ 허브 조회 자체 차단

# ===== 공통 경로 설정 =====
# Kaggle 환경 감지
IS_KAGGLE = os.path.exists('/kaggle/input')

# 대회 데이터 경로
DATA_PATH = '/kaggle/input/llm-classification-finetuning/' if IS_KAGGLE else './'

# 출력 경로
OUTPUT_PATH = '/kaggle/working/' if IS_KAGGLE else './'

print(f"📁 DATA_PATH: {DATA_PATH}")
print(f"💾 OUTPUT_PATH: {OUTPUT_PATH}")

# ============================================================================
# 📦 기본 설정
# ============================================================================

SEED = 42
N_SPLITS = 5
BATCH_SIZE = 32
MODEL_NAME = 'sentence-transformers/all-MiniLM-L6-v2'


📁 DATA_PATH: /kaggle/input/llm-classification-finetuning/
💾 OUTPUT_PATH: /kaggle/working/


In [2]:
import os, subprocess, glob, torch
from sentence_transformers import SentenceTransformer
import pandas as pd

ASSET_PATH = "/kaggle/input/llm-pref-assets-yechan-v1"  # 네가 만든 Dataset 이름으로 교체

# 후보 경로들
candidates = [
    f"{ASSET_PATH}/all-MiniLM-L6-v2",
    f"{ASSET_PATH}/c9745ed1d9f207416be6d2e6f8de32d1f16199bf",
]

# 없으면 zip 해제
if not any(os.path.exists(p) for p in candidates):
    zip_files = glob.glob(f"{ASSET_PATH}/*.zip")
    assert zip_files, f"모델 zip이 보이지 않음: {ASSET_PATH}"
    for z in zip_files:
        print("Unzipping:", z)
        subprocess.run(["unzip", "-q", z, "-d", ASSET_PATH], check=True)

# 모델 디렉토리 결정
candidates = [
    f"{ASSET_PATH}/all-MiniLM-L6-v2",
    f"{ASSET_PATH}/c9745ed1d9f207416be6d2e6f8de32d1f16199bf",
]
model_dir = next((p for p in candidates if os.path.exists(p)), None)
assert model_dir is not None, f"모델 디렉토리를 찾을 수 없음: {ASSET_PATH}"

print("📂 model_dir:", model_dir, "| files:", os.listdir(model_dir)[:10])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🔌 device:", device)

embedding_model = SentenceTransformer(
    model_dir,                              # ✅ '이름'이 아니라 '폴더 경로'
    device=device,
    local_files_only=True,                  # ✅ 허브 조회 차단
    tokenizer_kwargs={"chat_template": None}
)
print("✅ SentenceTransformer 로컬 로드 완료")


2025-11-01 09:04:21.983401: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761987862.166247      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761987862.221716      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


📂 model_dir: /kaggle/input/llm-pref-assets-yechan-v1/c9745ed1d9f207416be6d2e6f8de32d1f16199bf | files: ['config.json', '1_Pooling', 'README.md', 'tokenizer.json', 'tokenizer_config.json', 'sentence_bert_config.json', 'config_sentence_transformers.json', 'model.safetensors', 'modules.json', 'special_tokens_map.json']
🔌 device: cuda


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

✅ SentenceTransformer 로컬 로드 완료


In [3]:
# ============================================================================
# 📂 Cell 2: 데이터 로드
# ============================================================================

train_df = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
test_df = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))

print(f"Train: {len(train_df):,} rows")
print(f"Test: {len(test_df):,} rows")

# 타겟 변수 생성
target_cols = ['winner_model_a', 'winner_model_b', 'winner_tie']
train_df['winner'] = train_df[target_cols].idxmax(axis=1)
target_map = {'winner_model_a': 0, 'winner_model_b': 1, 'winner_tie': 2}
y = train_df['winner'].map(target_map)

print("\n타겟 분포:")
print(train_df['winner'].value_counts())

Train: 57,477 rows
Test: 3 rows

타겟 분포:
winner
winner_model_a    20064
winner_model_b    19652
winner_tie        17761
Name: count, dtype: int64


In [4]:
# ============================================================================
# 🔧 Cell 3: 특징 추출 함수
# ============================================================================
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler   # ✅ 추가

def extract_lexical_features(df):
    """어휘적 특징 추출"""
    df_copy = df.copy()

    # 길이 특징
    df_copy['len_a'] = df_copy['response_a'].str.len()
    df_copy['len_b'] = df_copy['response_b'].str.len()
    df_copy['len_diff'] = df_copy['len_a'] - df_copy['len_b']
    df_copy['len_ratio'] = df_copy['len_a'] / (df_copy['len_b'] + 1e-6)

    # 단어 수
    df_copy['words_a'] = df_copy['response_a'].str.split().str.len()
    df_copy['words_b'] = df_copy['response_b'].str.split().str.len()
    df_copy['words_diff'] = df_copy['words_a'] - df_copy['words_b']
    df_copy['words_ratio'] = df_copy['words_a'] / (df_copy['words_b'] + 1e-6)

    # 문장부호
    df_copy['puncs_a'] = df_copy['response_a'].str.count('[.!?]')
    df_copy['puncs_b'] = df_copy['response_b'].str.count('[.!?]')
    df_copy['puncs_diff'] = df_copy['puncs_a'] - df_copy['puncs_b']

    # 평균 단어 길이
    df_copy['avg_word_len_a'] = df_copy['len_a'] / (df_copy['words_a'] + 1)
    df_copy['avg_word_len_b'] = df_copy['len_b'] / (df_copy['words_b'] + 1)

    # 편향 특징
    df_copy['verbosity_ratio'] = df_copy['len_a'] / (df_copy['len_b'] + 1e-6)
    df_copy['is_a_longer'] = (df_copy['len_a'] > df_copy['len_b']).astype(int)
    df_copy['is_a_much_longer'] = (df_copy['len_a'] > df_copy['len_b'] * 1.5).astype(int)

    return df_copy

print("🔧 특징 추출 중...")
train_df = extract_lexical_features(train_df)
test_df = extract_lexical_features(test_df)

features = [
    'len_a', 'len_b', 'len_diff', 'len_ratio',
    'words_a', 'words_b', 'words_diff', 'words_ratio',
    'puncs_a', 'puncs_b', 'puncs_diff',
    'avg_word_len_a', 'avg_word_len_b',
    'verbosity_ratio', 'is_a_longer', 'is_a_much_longer'
]

X_train_lexical = train_df[features]
X_test_lexical = test_df[features]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_lexical)
X_test_scaled = scaler.transform(X_test_lexical)

print(f"어휘적 특징 형태: {X_train_scaled.shape}")

🔧 특징 추출 중...
어휘적 특징 형태: (57477, 16)


In [5]:
# ============================================================================
# 🧠 Cell 4: Sentence Transformers 임베딩
# ============================================================================

def get_embeddings(df, model, batch_size=BATCH_SIZE):
    """임베딩 생성"""
    sentences_a = (df['prompt'] + " [SEP] " + df['response_a']).tolist()
    sentences_b = (df['prompt'] + " [SEP] " + df['response_b']).tolist()

    print(f"  임베딩 생성 중 (총 {len(sentences_a) * 2:,}개)...")

    emb_a = model.encode(
        sentences_a,
        batch_size=batch_size,
        show_progress_bar=True,
        device=device,
        convert_to_numpy=True
    )

    emb_b = model.encode(
        sentences_b,
        batch_size=batch_size,
        show_progress_bar=True,
        device=device,
        convert_to_numpy=True
    )

    return emb_a, emb_b

print(f"🧠 임베딩 모델 재사용: {model_dir}")
# print(f"🧠 임베딩 모델 로드: {MODEL_NAME}")
# embedding_model = SentenceTransformer(
#     MODEL_NAME,
#     tokenizer_kwargs={"chat_template": None}
# )
# 이미 위에서 embedding_model을 만들었으면, 여기서는 재생성하지 말고 그대로 사용.
# 굳이 다시 만들 필요가 있으면 아래처럼 동일 옵션으로:
# embedding_model = SentenceTransformer(model_dir, device=device, local_files_only=True, tokenizer_kwargs={"chat_template": None})


print("📝 Train 임베딩 생성")
train_emb_a, train_emb_b = get_embeddings(train_df, embedding_model)

print("📝 Test 임베딩 생성")
test_emb_a, test_emb_b = get_embeddings(test_df, embedding_model)

print(f"\n임베딩 형태: {train_emb_a.shape}")


🧠 임베딩 모델 재사용: /kaggle/input/llm-pref-assets-yechan-v1/c9745ed1d9f207416be6d2e6f8de32d1f16199bf
📝 Train 임베딩 생성
  임베딩 생성 중 (총 114,954개)...


Batches:   0%|          | 0/1797 [00:00<?, ?it/s]

Batches:   0%|          | 0/1797 [00:00<?, ?it/s]

📝 Test 임베딩 생성
  임베딩 생성 중 (총 6개)...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


임베딩 형태: (57477, 384)


In [6]:
import numpy as np
# ============================================================================
# 🔗 Cell 5: 특징 결합
# ============================================================================

# 임베딩 특징 생성
emb_diff_train = train_emb_a - train_emb_b
emb_prod_train = train_emb_a * train_emb_b
emb_concat_train = np.hstack([train_emb_a, train_emb_b])

emb_diff_test = test_emb_a - test_emb_b
emb_prod_test = test_emb_a * test_emb_b
emb_concat_test = np.hstack([test_emb_a, test_emb_b])

# 모든 특징 결합
X_final_train = np.hstack([
    X_train_scaled,
    emb_diff_train,
    emb_prod_train,
    emb_concat_train
])

X_final_test = np.hstack([
    X_test_scaled,
    emb_diff_test,
    emb_prod_test,
    emb_concat_test
])

print(f"최종 특징 형태: {X_final_train.shape}")
print(f"  - 어휘적: {X_train_scaled.shape[1]}")
print(f"  - 임베딩 차이: {emb_diff_train.shape[1]}")
print(f"  - 임베딩 곱: {emb_prod_train.shape[1]}")
print(f"  - 임베딩 연결: {emb_concat_train.shape[1]}")




최종 특징 형태: (57477, 1552)
  - 어휘적: 16
  - 임베딩 차이: 384
  - 임베딩 곱: 384
  - 임베딩 연결: 768


In [7]:
# ============================================================================
# 🎯 Cell 6: 5-Fold 교차 검증
# ============================================================================

kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
oof_preds = np.zeros((len(train_df), 3))
log_losses = []
test_preds_list = []

print(f"🎯 {N_SPLITS}-Fold 교차 검증 시작...\n")

for fold, (train_idx, val_idx) in enumerate(kf.split(X_final_train, y), 1):
    print(f"📊 Fold {fold}/{N_SPLITS}")

    X_train, X_val = X_final_train[train_idx], X_final_train[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    # 모델 훈련
    model = LogisticRegression(
        random_state=SEED,
        C=0.1,
        max_iter=1000,
        solver='lbfgs',
        multi_class='multinomial',
        n_jobs=-1
    )

    model.fit(X_train, y_train)

    # 검증 예측
    val_preds = model.predict_proba(X_val)
    oof_preds[val_idx] = val_preds

    # 손실 계산
    loss = log_loss(y_val, val_preds)
    log_losses.append(loss)
    print(f"  Log Loss: {loss:.5f}")

    # 테스트 예측
    test_preds_list.append(model.predict_proba(X_final_test))
    print()

# 결과 요약
mean_log_loss = np.mean(log_losses)
std_log_loss = np.std(log_losses)

print("=" * 50)
print(f"🏆 평균 Log Loss: {mean_log_loss:.5f} ± {std_log_loss:.5f}")
print("=" * 50)

for i, loss in enumerate(log_losses, 1):
    print(f"Fold {i}: {loss:.5f}")


🎯 5-Fold 교차 검증 시작...

📊 Fold 1/5
  Log Loss: 1.05021

📊 Fold 2/5
  Log Loss: 1.05153

📊 Fold 3/5
  Log Loss: 1.04651

📊 Fold 4/5
  Log Loss: 1.04599

📊 Fold 5/5
  Log Loss: 1.04822

🏆 평균 Log Loss: 1.04849 ± 0.00212
Fold 1: 1.05021
Fold 2: 1.05153
Fold 3: 1.04651
Fold 4: 1.04599
Fold 5: 1.04822


In [8]:
# ============================================================================
# 📤 Cell 7: 제출 파일 생성
# ============================================================================

# 테스트 예측 평균
final_test_preds = np.mean(test_preds_list, axis=0)

# 제출 DataFrame 생성
submission_df = pd.DataFrame(final_test_preds, columns=target_cols)
submission_df['id'] = test_df['id']
submission_df = submission_df[['id'] + target_cols]

# 정규화 (확률 합 = 1)
row_sums = submission_df[target_cols].sum(axis=1)
submission_df[target_cols] = submission_df[target_cols].div(row_sums, axis=0)

# 검증
assert submission_df['id'].equals(test_df['id']), "❌ ID 순서 불일치!"
assert np.allclose(submission_df[target_cols].sum(axis=1), 1.0), "❌ 확률 합이 1이 아님!"
assert (submission_df[target_cols] >= 0).all().all(), "❌ 음수 확률 존재!"

print("✅ 검증 통과!")
# 저장
submission_path = os.path.join(OUTPUT_PATH, 'submission.csv')
submission_df.to_csv(submission_path, index=False)

print(f"\n✅ {submission_path} 생성 완료!")
print(f"\n제출 파일 미리보기:")
print(submission_df.head(10))
print(f"\n통계:")
print(submission_df[target_cols].describe())

print("\n🎉 훈련 완료!")

✅ 검증 통과!

✅ /kaggle/working/submission.csv 생성 완료!

제출 파일 미리보기:
        id  winner_model_a  winner_model_b  winner_tie
0   136060        0.241380        0.234171    0.524449
1   211333        0.406659        0.324416    0.268924
2  1233961        0.253437        0.391925    0.354638

통계:
       winner_model_a  winner_model_b  winner_tie
count        3.000000        3.000000    3.000000
mean         0.300492        0.316837    0.382671
std          0.092141        0.079150    0.130048
min          0.241380        0.234171    0.268924
25%          0.247409        0.279294    0.311781
50%          0.253437        0.324416    0.354638
75%          0.330048        0.358171    0.439544
max          0.406659        0.391925    0.524449

🎉 훈련 완료!
